In [1]:
import cv2
import os
import numpy as np
import torch
import torch.optim as optim
from torch import nn
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from matplotlib.backends.backend_agg import FigureCanvasAgg as FigureCanvas
from PIL import Image
from segmentation_models_pytorch.losses import DiceLoss

In [2]:
if torch.cuda.is_available():
    device = torch.device("cuda:2")
elif torch.backends.mps.is_available():
    device = torch.device("mps")
else:
    device = torch.device("cpu")
print(f"using device: {device}")

# отключили автокаст, тк бекворд не работает с ним
# if device.type == "cuda":
#     # use bfloat16 for the entire notebook
#     torch.autocast("cuda", dtype=torch.bfloat16).__enter__()
#     # turn on tfloat32 for Ampere GPUs (https://pytorch.org/docs/stable/notes/cuda.html#tensorfloat-32-tf32-on-ampere-devices)
#     if torch.cuda.get_device_properties(0).major >= 8:
#         torch.backends.cuda.matmul.allow_tf32 = True
#         torch.backends.cudnn.allow_tf32 = True
# elif device.type == "mps":
#     print(
#         "\nSupport for MPS devices is preliminary. SAM 2 is trained with CUDA and might "
#         "give numerically different outputs and sometimes degraded performance on MPS. "
#         "See e.g. https://github.com/pytorch/pytorch/issues/84936 for a discussion."
#     )

using device: cuda:2


In [30]:
# np.random.seed(3)

# def show_mask(mask, ax, random_color=False, borders = False):
#     if random_color:
#         color = np.concatenate([np.rZandom.random(3), np.array([0.6])], axis=0)
#     else:
#         color = np.array([30/255, 144/255, 255/255, 0.6])
#     h, w = mask.shape[-2:]
#     mask = mask.astype(np.uint8)
#     mask_image =  mask.reshape(h, w, 1) * color.reshape(1, 1, -1)
#     if borders:
#         import cv2
#         contours, _ = cv2.findContours(mask,cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE) 
#         # Try to smooth contours
#         contours = [cv2.approxPolyDP(contour, epsilon=0.01, closed=True) for contour in contours]
#         mask_image = cv2.drawContours(mask_image, contours, -1, (1, 1, 1, 0.5), thickness=2) 
#     ax.imshow(mask_image)

# def show_points(coords, labels, ax, marker_size=375):
#     pos_points = coords[labels==1]
#     neg_points = coords[labels==0]
#     ax.scatter(pos_points[:, 0], pos_points[:, 1], color='green', marker='*', s=marker_size, edgecolor='white', linewidth=1.25)
#     ax.scatter(neg_points[:, 0], neg_points[:, 1], color='red', marker='*', s=marker_size, edgecolor='white', linewidth=1.25)   

# def show_box(box, ax):
#     x0, y0 = box[0], box[1]
#     w, h = box[2] - box[0], box[3] - box[1]
#     ax.add_patch(plt.Rectangle((x0, y0), w, h, edgecolor='green', facecolor=(0, 0, 0, 0), lw=2)) 

# def show_box(box, image):
#     x0, y0 = box[0], box[1]
#     w, h = box[2] - box[0], box[3] - box[1]
#     image_box = cv2.rectangle(image, (int(x0), int(y0)), (int(x0 + w), int(y0 + h)), (255, 0, 0), 2)
#     return image_box, image

# def show_masks(image, masks, scores, point_coords=None, box_coords=None, input_labels=None, borders=False):
#     for i, (mask, score) in enumerate(zip(masks, scores)):
#         plt.figure(figsize=(10, 10))
#         plt.imshow(image)
#         show_mask(mask, plt.gca(), borders=borders)
#         if point_coords is not None:
#             assert input_labels is not None
#             show_points(point_coords, input_labels, plt.gca())
#         if box_coords is not None:
#             # boxes
#             show_box(box_coords, plt.gca())
#         if len(scores) > 1:
#             plt.title(f"Mask {i+1}, Score: {score:.3f}", fontsize=18)
#         plt.axis('off')
#         plt.show()

# def show_mask(mask_n, image):
#     print(mask_n.shape)
#     print(image.shape)
#     image_masked = cv2.addWeighted(image, 0.5, mask_n, 0.5, 0)
#     return image_masked

# def show_mask(mask, image):
#     image_n = image.copy(deep = True)
#     mask_n = mask.copy(deep = True)
#     image_n[mask_n == 255] = (0, 255, 0)
#     return image_n, image

def save_mask(image, mask, box, iter, alpha=0.7):
    image = (np.array(image) / 255.).transpose(2, 0, 1)
    mask = np.stack([mask[0] * 87./255, mask[0] * 186./255, mask[0] * 168./255])

    image_transparency = (image * (1 - alpha) + mask * alpha).clip(0, 1)

    image = np.where(mask, image_transparency, image).clip(0, 1)
    image = Image.fromarray((image.transpose(1, 2, 0) * 255).astype(dtype=np.uint8))
    
    x0, y0 = box[0], box[1]
    w, h = box[2] - box[0], box[3] - box[1]
    image_box = cv2.rectangle(image, (int(x0), int(y0)), (int(x0 + w), int(y0 + h)), (255, 0, 0), 2)
    
    image.save('./bbox_frames/' + str(iter).zfill(4) + '.jpg')
    
    
# def save_masks(image, masks, box_coords=None, borders=False):
#     fig = plt.figure(figsize=(image.shape[1]/100, image.shape[0]/100))
#     plt.imshow(image)
#     show_mask(mask, plt.gca(), borders=borders)
#     if box_coords is not None:
#         show_box(box_coords, plt.gca())
#     plt.axis('off')
    
#     # fig = plt.gcf()
#     fig.canvas.draw()
#     canvas = FigureCanvas(fig)
#     canvas.draw()
#     plt.close()
#     img = np.array(fig.canvas.renderer.buffer_rgba())[:,:,:3]
#     img = cv2.cvtColor(np.uint8(img*255), cv2.COLOR_RGB2BGR)
#     # print(img.shape)
#     plt.clf()
#     return img

# def save_masks(image, mask, box_coords):
#     image_mask, image = show_mask(mask, image)
#     cv2.imwrite('./bbox_frames/' + str(iter).zfill(4) + 'image1' +'.jpg', image)
#     img_box, image = show_box(box_coords, image_mask)
#     cv2.imwrite('./bbox_frames/' + str(iter).zfill(4) + 'image2' +'.jpg', image)
#     return img_box, image

In [17]:
def square (coords):
    return (coords[2] - coords[0]) * (coords[3] - coords[1])

In [18]:
image = Image.open('../images/GrabCut/data_GT/sheep.jpg')
image = np.array(image.convert("RGB"))

In [19]:
h, w = image.shape[:2]
print(h,w)

600 450


In [20]:
zero_mask = Image.open('../images/GrabCut/boundary_GT/sheep.bmp')
zero_mask = np.array(zero_mask)
zero_mask = torch.as_tensor(zero_mask, dtype=torch.float)

In [21]:
from sam2.build_sam import build_sam2
from sam2.sam2_image_predictor import SAM2ImagePredictor

sam2_checkpoint = "../../checkpoints/sam2_hiera_large.pt"
model_cfg = "sam2_hiera_l.yaml"

sam2_model = build_sam2(model_cfg, sam2_checkpoint, device=device)

predictor = SAM2ImagePredictor(sam2_model)

# убираем отслеживание всех параметров
for param in predictor.model.parameters():
    param.requires_grad = False

In [22]:
predictor.set_image(image)

In [23]:
input_box = np.array([177, 182, 296, 378])

In [24]:
# зачем
mask_input, unnorm_coords, labels, unnorm_box_input = predictor._prep_prompts(
    point_coords=None, point_labels=None, box=input_box, mask_logits=None, normalize_coords=True
)

In [25]:
unnorm_box_input

tensor([[[402.7733, 310.6133],
         [673.5645, 645.1200]]], device='cuda:2')

In [26]:
# boxes_n = torch.as_tensor(input_box, dtype=torch.float, device=predictor.device)
# boxes = boxes_n.reshape(-1, 2, 2)
# coords = boxes.clone()
# coords[..., 0] = coords[..., 0] / w
# print(coords)
# coords[..., 1] = coords[..., 1] / h
# print(coords)
# coords = coords * 1024  # хз почему, мб макс разрешение sam2
# print(coords)

In [27]:
# делаем обратную нормализацию координат
def back_norm(unnorm_box, w, h):
    norm_box = unnorm_box/1024
    coords = norm_box.clone()
    coords[..., 0] = coords[..., 0] * w
    coords[..., 1] = coords[..., 1] * h
    box_coord = coords.squeeze(0).float().detach().cpu().numpy().flatten()
    return box_coord

In [28]:
# criterion = nn.MSELoss()
criterion = DiceLoss('binary') 

In [31]:
iters = 200
lambda_reg = 0.01

unnorm_box = unnorm_box_input
unnorm_box.requires_grad = True
opt = optim.Adam([unnorm_box], lr=1e-2)
    
for iter in range(iters):
    opt.zero_grad()
    
    masks, iou_predictions, low_res_masks = predictor._predict(
        unnorm_coords,
        labels,
        unnorm_box,
        mask_input,
        multimask_output=False,
        return_logits=True
    )

    masks_np = (masks>0.0).squeeze(0).detach().cpu().numpy()*255
    iou_predictions_np = iou_predictions.squeeze(0).float().detach().cpu().numpy()
    low_res_masks_np = low_res_masks.squeeze(0).float().detach().cpu().numpy()
    unnorm_coords_np = back_norm(unnorm_box, w, h)

    masks_np2 = masks_np.squeeze(0)
    masks_loss = masks.cpu()
    zero_mask_torch = zero_mask.unsqueeze(0).cpu()

    save_mask(image: Image, mask: np.ndarray, unnorm_coords_np, iter, alpha=0.7):
    
    # loss = criterion(masks.squeeze().cpu(), zero_mask)
    # print(masks)
    # print(zero_mask_torch)
    loss = criterion(masks_loss, zero_mask_torch)
    reg_loss = abs(square(input_box)-square(unnorm_coords_np))
    total_loss = loss + lambda_reg * reg_loss**2
    
    if iter % 100 == 0:
        print(loss)
    
    total_loss.backward()
    opt.step()

SyntaxError: invalid syntax (3717263736.py, line 29)

In [ ]:
!ffmpeg -i /home/user20/segment-anything-2/notebooks/sam2_adversarial_attacks/bbox_frames/%04d.jpg -r 30 bbox_video/out17.mp4